In [3]:
# include("src/JuTrack.jl")

Main.JuTrack

In [5]:
using .JuTrack
using Enzyme
using Test
using BenchmarkTools
include("test/ssrf_ring.jl")

ssrf (generic function with 1 method)

In [21]:
#CTPS = CTPS (intial value, what palce in array that derv occupies, # of unique variables finding derv of, order of calculation)

In [64]:
function fodo_ring(K1, RAD)
        D1 = DRIFT(name="D1", len=0.34)
        D2 = DRIFT(name="D2", len=0.59)
        QF1 = KQUAD(name="QF1", len=0.32, k1=K1, rad=RAD )
        QD1 = KQUAD(name="QD1", len=0.32, k1=-1.192160, rad=RAD  )
        BendingAngle = pi/20
        BD1 = SBEND(name="BD1", len=0.72, angle=BendingAngle/2, e1=BendingAngle/2, e2=0.0 , rad=RAD )
        BD2 = SBEND(name="BD2", len=0.72, angle=BendingAngle/2, e1=0.0, e2=BendingAngle/2, rad=RAD  )

        FODO = (QF1, D1, QD1, D1, QF1)
        BD1_comp = (BD1)
        BD2_comp = (BD2)
        D2_comp = (D2)

        CELL = Vector{AbstractElement}(undef, 8)
        for i in eachindex(FODO)
                CELL[i] = FODO[i]
        end
        CELL[length(FODO)+1] = BD1
        CELL[length(FODO)+2] = D2
        CELL[length(FODO)+3] = BD2
                # CELL[i+172] = M5[i]
        
        ELIST = Vector{AbstractElement}(undef, 4*length(CELL))
        for i in eachindex(CELL)
                ELIST[i] = CELL[i]
                ELIST[i+length(CELL)] = CELL[i]
                ELIST[i+2*length(CELL)] = CELL[i]
                ELIST[i+3*length(CELL)] = CELL[i]
        end
        return ELIST
end

function test_track(k1_in)
        particles = [0.001 0.0001 0.0005 0.0002 0.0 0.0; 0.001 0.0 0.0 0.0 0.0 0.0]
        beam = Beam(particles)
        line = fodo_ring(k1_in[1],0.0)
        linepass!(line, beam)
        return beam.r[1,1]
end

function test_track_TPS(k1_in)
        line = fodo_ring(k1_in[1],0.0)
        x = CTPS(0.0, 1, 6, 3)
        xp = CTPS(0.0, 2, 6, 3)
        y = CTPS(0.0, 3, 6, 3)
        yp = CTPS(0.0, 4, 6, 3)
        delta = CTPS(0.0, 5, 6, 3)
        z = CTPS(0.0, 6, 6, 3)
        rin = [x, xp, y, yp, delta, z]
        linepass_TPSA!(line, rin)
        return rin[1].map
end

test_track_TPS (generic function with 2 methods)

In [65]:

println(test_track_TPS(-1.063770))
# println(length(test_track_TPS(-1.063770)))
# x = [-1.063770]
# println(test_track_TPS(x))
# @btime test_track_TPS(x) 
# @btime grad = gradient(Forward, test_track_TPS, x)

[0.0, 1496.2906690330187, 1924.4239164767187, 0.0, 0.0, 0.0, 93.8640867724126, 0.0, 0.0, 0.0, 0.0, 0.0, -5567.960057661321, 0.0, 0.0, 0.0, 0.0, -7930.821865766962, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -312.43542475121046, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14330.827348699735, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21845.004649950602, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 740.2241567706814]


In [44]:

println(test_track(-1.063770))
x = [-1.063770]
println(test_track(x))
@btime test_track(x) 
@btime grad = gradient(Forward, test_track, x)


1.0262208293145973
1.0262208293145973
  88.900 μs (698 allocations: 79.30 KiB)
  502.400 μs (3126 allocations: 163.05 KiB)


(-2.2295173605244214,)